In [1]:
from train_contrastive import load_data

In [8]:
path = "../data/tabula_muris/preprocessed_reduced/tm_adata_test.pkl"

X, domains, cells, adata = load_data(path)

In [9]:
from contrastive_model import ContrastiveModel

c_transformer_path = "runs/attention_05-10-20:43/final.ckpt"

c_transformer = ContrastiveModel.load_from_checkpoint(c_transformer_path)

In [10]:
import torch

cuda_device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Set tensors to the appropriate device
x = torch.tensor(X, dtype=torch.float32).to(cuda_device)

c_transformer.to(cuda_device)

ContrastiveModel(
  (encoder): AttentionEncoder(
    (expr_proj): Linear(in_features=1, out_features=512, bias=True)
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-7): 8 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (projection_head): ProjectionHead(
      (net): Sequential(
        (0): Linear(in_features=512, out_features=2048, bias=True)
        (1): 

In [11]:
import torch
from tqdm import tqdm

batch_size = 32  # Define a suitable batch size
ct_embeddings_list = []

c_transformer.eval()

with torch.no_grad():
    for i in tqdm(range(0, x.size(0), batch_size), desc="Processing Batches"):
        x_r_batch = x[i:i + batch_size]
        
        ct_embeddings_batch = c_transformer.encoder(x_r_batch)
        
        ct_embeddings_list.append(ct_embeddings_batch)

ct_embeddings = torch.cat(ct_embeddings_list, dim=0)

Processing Batches: 100%|███████████████████████████████████████████████████████| 1931/1931 [01:26<00:00, 22.34it/s]


In [12]:
adata.obsm["ct_embeddings"] = ct_embeddings.cpu().numpy()

/tmp/ipykernel_2571819/1969382214.py:1: ImplicitModificationWarning: Setting element `.obsm['ct_embeddings']` of view, initializing view as actual.
  adata.obsm["ct_embeddings"] = ct_embeddings.cpu().numpy()


In [13]:
import scanpy as sc

sc.pp.neighbors(adata, use_rep="ct_embeddings")  # Compute neighbors using latent space
sc.tl.umap(adata)
fig1 = sc.pl.umap(adata, color='cell_ontology_class', legend_loc=None, title='', frameon=False, size=50, return_fig=True, show=True)

/oscar/rt/9.2/software/0.20-generic/0.20.1/opt/spack/linux-rhel9-x86_64_v3/gcc-11.3.1/anaconda-2023.09-0-7nso27ys7navjquejqdxqylhg7kuyvxo/lib/python3.11/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [19]:
import matplotlib.pyplot as plt
import pandas as pd

# Extract the necessary data
cell_ontology_classes = adata.obs['cell_ontology_class']
tech_types = adata.obs['tech']

# Create a DataFrame for easier manipulation
df = pd.DataFrame({'cell_ontology_class': cell_ontology_classes, 'tech': tech_types})

# Group by 'cell_ontology_class' and 'tech' to count occurrences
grouped = df.groupby(['cell_ontology_class', 'tech']).size().unstack(fill_value=0)

# Plot a bar graph for each category in 'cell_ontology_class' with bars side by side
grouped.plot(kind='bar', stacked=False, figsize=(12, 8))

plt.title('Number of Examples with Each "Tech" Type per Cell Ontology Class')
plt.xlabel('Cell Ontology Class')
plt.ylabel('Number of Examples')
plt.legend(title='Tech Type')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('analysis1.png')


In [23]:
# Filter the data to include only "basal cell of epidermis"
basal_cells = adata[adata.obs['cell_ontology_class'] == 'skeletal muscle satellite cell']

# Plot UMAP, coloring by 'tech'
fig2 = sc.pl.umap(basal_cells, color='tech', legend_loc='right margin', title='UMAP of Basal Cells of Epidermis by Tech', frameon=False, size=50, return_fig=True, show=False)
fig2.savefig('analy2.png')


/users/ndepiero/.local/lib/python3.11/site-packages/scanpy/plotting/_utils.py:487: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[value_to_plot + "_colors"] = colors_list


In [26]:
# Get the top 10 most occurring cell_ontology_classes
top_10_classes = adata.obs['cell_ontology_class'].value_counts().nlargest(10).index

# Filter the adata to include only the top 10 classes
filtered_adata = adata[adata.obs['cell_ontology_class'].isin(top_10_classes)]

# Plot UMAP for the filtered data
fig1_top10 = sc.pl.umap(filtered_adata, color='cell_ontology_class', legend_loc=None, title='Top 10 Cell Ontology Classes', frameon=False, size=50, return_fig=True, show=True)
fig1_top10.savefig('analy3.png')